In [1]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections
from pyspark.ml.linalg import Vectors
from sklearn.cluster import MiniBatchKMeans, KMeans
import numpy as np

In [2]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [3]:
inputLines = spark.sparkContext.textFile("train_full_parsed_clean5.1.csv")

In [4]:
header=inputLines.filter(lambda l: "_id" in l)

In [5]:
header.collect()

['log_duration,log_trip_duration,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,distance,speed_great_circle,hour_count,snow,holiday,vendor_id,pickup_hour_0,pickup_hour_1,pickup_hour_2,pickup_hour_3,pickup_hour_4,pickup_hour_5,pickup_hour_6,pickup_hour_7,pickup_hour_8,pickup_hour_9,pickup_hour_10,pickup_hour_11,pickup_hour_12,pickup_hour_13,pickup_hour_14,pickup_hour_15,pickup_hour_16,pickup_hour_17,pickup_hour_18,pickup_hour_19,pickup_hour_20,pickup_hour_21,pickup_hour_22,pickup_hour_23,pickup_weekday_0,pickup_weekday_1,pickup_weekday_2,pickup_weekday_3,pickup_weekday_4,pickup_weekday_5,pickup_weekday_6,pickup_month_1,pickup_month_2,pickup_month_3,pickup_month_4,pickup_month_5,pickup_month_6,passenger_count_0,passenger_count_1,passenger_count_2,passenger_count_3,passenger_count_4,passenger_count_5,passenger_count_6,passenger_count_7,route_cooridnates']

In [6]:
linesnoheader = inputLines.subtract(header)

In [7]:
linesnoheader.take(2)

['6.274385302458052,6.061456918928017,-74.00108337402342,40.7391014099121,-73.98416137695312,40.72916030883789,1804.6226836023934,2067.3,15.179069301328546,795,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,"[[-74.001057, 40.739136], [-73.990717, 40.734794], [-73.991462, 40.731745], [-73.98444, 40.728783], [-73.984159, 40.729159]]"',
 '6.366126746061101,6.71901315438526,-74.0135040283203,40.713882446289055,-73.99567413330078,40.72507095336913,1951.3708639798651,2991.4,8.494480181774502,3712,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,"[[-74.013611, 40.713904], [-74.012759, 40.717114], [-74.011235, 40.724276], [-74.008441, 40.724022], [-74.004618, 40.72137], [-73.999962, 40.726892], [-73.9971, 40.725462], [-73.995661, 40.725101]]"']

In [8]:
coord=pd.read_csv('newcounter.csv')
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000,random_state=1234).fit(coord[['longitude','latitude']])
coord.loc[:, 'cluster'] = kmeans.predict(coord[['longitude','latitude']])

speed_cluster=pd.read_csv("hour_speed_cluster.csv")

In [9]:
cluster=coord['cluster'].tolist()

cluster_name=['C'+str(i) for i in range(100)]


In [10]:
def topcoordinates(line):
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)

    line_cluster=[]
    for i in coordinates:
                temp=re.split('\[|, |\]',i)
                long=float(temp[1])
                lat=float(temp[2])
                cluster_group=kmeans.predict([[long,lat]])
                line_cluster+=[cluster_group[0]]

    line_cluster=list(set(line_cluster))
    feature=line.split(',')
    all_cluster=[ float(speed_cluster.loc[speed_cluster.hour_count==int(feature[9]) , 'C'+str(i)].values[0]) if i in line_cluster else np.nan for i in range(100)]





    feature1=[float(i) for i in feature[:58]]
    feature2=[int(i) for i in feature1[10:58]]
    feature3=feature1[:8]
    return feature3+feature2+all_cluster
    

In [14]:
line='6.274385302458052,6.061456918928017,-74.00108337402342,40.7391014099121,-73.98416137695312,40.72916030883789,1804.6226836023934,2067.3,15.179069301328546,795,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,"[[-74.001057, 40.739136], [-73.990717, 40.734794], [-73.991462, 40.731745], [-73.98444, 40.728783], [-73.984159, 40.729159]]"'
coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    
line_cluster=[]
for i in coordinates:
            temp=re.split('\[|, |\]',i)
            long=float(temp[1])
            lat=float(temp[2])
            cluster_group=kmeans.predict([[long,lat]])
            line_cluster+=[cluster_group[0]]

line_cluster=list(set(line_cluster))
feature=line.split(',')
all_cluster=[ float(speed_cluster.loc[speed_cluster.hour_count==int(feature[9]) , 'C'+str(i)].values[0]) if i in line_cluster else np.nan for i in range(100)]
        
    
    
    

feature1=[float(i) for i in feature[:58]]
feature2=[int(i) for i in feature1[10:58]]
feature3=feature1[:8]

In [21]:
len(feature3+feature2+all_cluster)

156

In [11]:
coord = linesnoheader.map(topcoordinates)

In [12]:
feature=['log_duration', 'log_trip_duration', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance','distance',
       'snow', 'holiday', 'vendor_id', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11',
       'pickup_hour_12', 'pickup_hour_13', 'pickup_hour_14',
       'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23',
       'pickup_weekday_0', 'pickup_weekday_1', 'pickup_weekday_2',
       'pickup_weekday_3', 'pickup_weekday_4', 'pickup_weekday_5',
       'pickup_weekday_6', 'pickup_month_1', 'pickup_month_2',
       'pickup_month_3', 'pickup_month_4', 'pickup_month_5',
       'pickup_month_6', 'passenger_count_0', 'passenger_count_1',
       'passenger_count_2', 'passenger_count_3', 'passenger_count_4',
       'passenger_count_5', 'passenger_count_6', 'passenger_count_7']
colNames=feature+cluster_name

In [13]:
colNames

['log_duration',
 'log_trip_duration',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'great_circle_distance',
 'distance',
 'snow',
 'holiday',
 'vendor_id',
 'pickup_hour_0',
 'pickup_hour_1',
 'pickup_hour_2',
 'pickup_hour_3',
 'pickup_hour_4',
 'pickup_hour_5',
 'pickup_hour_6',
 'pickup_hour_7',
 'pickup_hour_8',
 'pickup_hour_9',
 'pickup_hour_10',
 'pickup_hour_11',
 'pickup_hour_12',
 'pickup_hour_13',
 'pickup_hour_14',
 'pickup_hour_15',
 'pickup_hour_16',
 'pickup_hour_17',
 'pickup_hour_18',
 'pickup_hour_19',
 'pickup_hour_20',
 'pickup_hour_21',
 'pickup_hour_22',
 'pickup_hour_23',
 'pickup_weekday_0',
 'pickup_weekday_1',
 'pickup_weekday_2',
 'pickup_weekday_3',
 'pickup_weekday_4',
 'pickup_weekday_5',
 'pickup_weekday_6',
 'pickup_month_1',
 'pickup_month_2',
 'pickup_month_3',
 'pickup_month_4',
 'pickup_month_5',
 'pickup_month_6',
 'passenger_count_0',
 'passenger_count_1',
 'passenger_count_2',
 'passenger_count_3',
 'passen

In [16]:
train = coord.toDF(colNames)

In [ ]:
train.write.csv("train_full_parsed_clean5.csv",header = 'true')

In [ ]:
train.repartition(1).write.format('com.databricks.spark.csv').save('csv_cluster.csv',header = 'true')

In [ ]:
train.repartition(40).write.parquet('parquet_cluster_40_files_v2')

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder

In [ ]:
feature_col=colNames[0:1]+colNames[2:]
vecAssembler = VectorAssembler(inputCols=feature_col, outputCol="features")


In [ ]:
(trainingData, testData) = train.randomSplit([0.8, 0.2],seed=42)

In [ ]:

gbt = GBTRegressor(featuresCol="features",labelCol="log_trip_duration",maxIter=10,maxDepth=10, seed=42)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[vecAssembler,gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [ ]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

model = GradientBoostedTrees.trainRegressor(trainingData,
                                             categoricalFeaturesInfo={}, numIterations=3)